<a href="https://colab.research.google.com/github/JuniorHZ19/ML/blob/main/perceptronmulticapa5.6mejorar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch torchvision

In [13]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
import numpy as np


class MiDataSet(Dataset):

  def __init__(self,csv_file):

     data=pd.read_csv(csv_file)

     self.x,self.y= self.limpiezaDatos(data)

     self.samples=self.y.shape[0]


  def __getitem__(self,id):

   return self.x[id],self.y[id]


  def __len__(self):
   return self.samples


  def limpiezaDatos(self,data):

    data=data.dropna(how="all")

    data=data.drop_duplicates()

    data=data.drop(columns=["name","ticket","cabin","boat","body","home.dest"])

    data[["age","fare"]]=data[["age","fare"]].fillna(data[["age","fare"]].mean())

    data["embarked"]=data["embarked"].fillna("S")

    data["age"]=data["age"].astype(int)

    data=pd.get_dummies (data,columns=["sex","embarked"],prefix=["sex","embarked"])

    variables_x=data.drop(columns=["survived"])
    print(variables_x.head())
    variable_y=data["survived"]

    return variables_x.values,  variable_y.values







In [45]:
import sklearn

from sklearn.model_selection import train_test_split

mis_datos=MiDataSet("/content/titanic.csv")

#Estandarizar los datos solo los datos que son conitunos no los categoricos como dumies que tiene 0-1 etc

columnas_continuas = [1,4] #columnas no categoricas 1:age 2:fare



def normalizar_datos(data,columnas_no_cat,mean_value_continuos=None,std_value_continuos=None):
 datos_continuos = data[:, columnas_no_cat] #tomaremos los datos de nuestro dataset de esas columnas

 if mean_value_continuos is  None:
  mean_value_continuos = np.mean(datos_continuos, axis=0) #Calculamos el mean de esas columnas

 if std_value_continuos is  None:
  std_value_continuos = np.std(datos_continuos, axis=0) #Calculamos el desviacion  estadar de esas columnas

 normalized_data_continuos = (datos_continuos - mean_value_continuos) / std_value_continuos #normalizaoms con la desviacion y desviacion estanr cada dato
 data[:, columnas_no_cat] = normalized_data_continuos #regresmos los datos de las columnas  no categoricas normalizadas de nuevo a los datos

 return data,mean_value_continuos,std_value_continuos


data_x_normalizada,mean_,std_=normalizar_datos(mis_datos.x,columnas_continuas)

print(mean_)

X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(data_x_normalizada, mis_datos.y, test_size=0.2, random_state=0,stratify=mis_datos.y)

print(X_entrenamiento.shape)

print(X_prueba)

print(y_entrenamiento.shape)

print(y_prueba)




   pclass  age  sibsp  parch      fare  sex_female  sex_male  embarked_C  \
0       1   29      0      0  211.3375           1         0           0   
1       1    0      1      2  151.5500           0         1           0   
2       1    2      1      2  151.5500           1         0           0   
3       1   30      1      2  151.5500           0         1           0   
4       1   25      1      2  151.5500           1         0           0   

   embarked_Q  embarked_S  
0           0           1  
1           0           1  
2           0           1  
3           0           1  
4           0           1  
[29.68525592 33.29547928]
(1047, 10)
[[ 2.          0.3346087   1.         ...  0.          0.
   1.        ]
 [ 2.         -0.44089198  0.         ...  0.          0.
   1.        ]
 [ 3.         -0.28579185  0.         ...  0.          1.
   0.        ]
 ...
 [ 2.          0.3346087   0.         ...  0.          0.
   1.        ]
 [ 3.         -2.14699347  4.         ...

In [34]:



from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler


conjunto_entrenamiento = TensorDataset( torch.tensor(X_entrenamiento).float(), torch.tensor(y_entrenamiento).float())

conjunto_prueba = TensorDataset( torch.tensor(X_prueba).float(),  torch.tensor(y_prueba).float())

data_por_lote_entrenamiento=DataLoader(conjunto_entrenamiento,batch_size=10,shuffle=True)

data_por_por_lote_test=DataLoader(conjunto_prueba,batch_size=10,shuffle=True)



#datos_features_train,datos_label_train=next(iter(data_por_lote_entrenamiento))

#datos_features_test,datos_label_test=next(iter(data_por_por_lote_test))


print(data_por_lote_entrenamiento.batch_size)




10


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

modelo=torch.Sequential(

                        torch.nn.Linear(input_size,3),
                        torch.nn.ReLU(),

                        torch.nn.Linear(3,4),
                        torch.nn.ReLU(),

                        torch.nn.Linear(4,1),
                        torch.nn.Sigmoid()
)

In [31]:
import torch.nn as nn
import torch.nn.functional as F



class PerceptronMulticapa(nn.Module):

   def __init__(self,input_size):
    super(PerceptronMulticapa,self).__init__()
    print(input_size)

    self.fc1=nn.Linear(input_size,3)

    self.fc2=nn.Linear(3,4)

    self.capa_salida =nn.Linear(4,1)

   def forward(self,x):

     x=self.fc1(x)
     x=F.relu(x)

     x=self.fc2(x)
     x=F.relu(x)

     output=self.capa_salida(x)



     return output


In [32]:

if torch.cuda.is_available():
    print("CUDA está disponible.")
    device = torch.device("cuda")
else:
    print("CUDA no está disponible. Se utilizará la CPU.")
    device = torch.device("cpu")

CUDA está disponible.


In [ ]:
import numpy as np


#funcion clasificador:
def clasificador(y_pred_test):

  umbral = 0.6
  y_pred_test_binario = (y_pred_test >= umbral).to(torch.float32)
  return y_pred_test_binario

#funcion acutity:

def accuracy(y_test, y_real):
    correctos = torch.sum(y_test == y_real).item()

    return (correctos / len(y_test))


import torch.optim as optim



def TrainMLP(modelo,epochs,optimisador,criterio,input_size,data_Loader_entrenamiento,data_Loader_test):

 for epoch in range(epochs):

  total_train_loss = 0.0
  num_train_batches = len(data_Loader_entrenamiento)
  modelo.train()

  for x_entrenamiento,y_entrenamiento in (data_Loader_entrenamiento):
   x_entrenamiento,y_entrenamiento=x_entrenamiento.to(device),y_entrenamiento.to(device)

   y_predict=modelo(x_entrenamiento)

   loss=criterio(y_predict,y_entrenamiento.view(-1,1))

   loss.backward()
   optimisador.step()
   optimisador.zero_grad()
   total_train_loss+=loss.item()

  modelo.eval()
  with torch.no_grad():
    total_test_accuracy = 0.0
    num_test_batches = len(data_Loader_test)

    for X_prueba,y_prueba in (data_Loader_test):

     X_prueba,y_prueba=X_prueba.to(device),y_prueba.to(device)
     y_predict_test=modelo(X_prueba)
     y_predict_test_p=torch.sigmoid(y_predict_test)
     y_predic_test_clasificado=clasificador(y_predict_test_p)
     Precision=accuracy(y_predic_test_clasificado,y_prueba.view(-1,1))
     total_test_accuracy += Precision

  avg_test_accuracy = (total_test_accuracy / num_test_batches)
  avg_train_loss=(total_train_loss/num_train_batches)
  print(f'Epoca[{epoch+1}], Perdida: {avg_train_loss:.4f},Accuryty:{avg_test_accuracy:.4f}')





#-----------------------------------------------------------------------------------------------

input_sizes=10

PerceptronMulti=PerceptronMulticapa(input_sizes)

criterios=nn.BCEWithLogitsLoss() # automaticamente aplica el sigmoide a la capa de salida y clasfica 0 -1 para el loss

optimizer=optim.SGD(PerceptronMulti.parameters(),lr=0.01)

epocas=500

PerceptronMulti=PerceptronMulti.to(device)

TrainMLP(PerceptronMulti,epocas,optimizer,criterios,input_sizes,data_por_lote_entrenamiento,data_por_por_lote_test)







In [47]:
#nuevo datos:


nuevo_x=[[1,85,0,0,55.155,0,0,0,1,0]]

coulmans=[1,4]
data_x_normalizada,_,_=normalizar_datos(nuevo_x,coulmans,mean_,std_)

nuevo_X_transform=torch.from_numpy(data_x_normalizada.transform(nuevo_x)).float()
print(nuevo_X_transform)
y_predict=PerceptronMulti(nuevo_X_transform)

print(clasificador(y_predict))



TypeError: ignored

In [ ]:


y_pred_multiclases = torch.tensor([[0.1, 1.8, 0.8, 0.4, 2],
                                   [0.5, 2, 3.5, 5, 2.1]])

def clasificador(y_pred_test):

   max_values_tensor = torch.empty((0,), dtype=torch.float32)

   for tensor in y_pred_test:

     max_values, max_indices  = torch.max(tensor, dim=0)

     max_values_tensor = torch.cat((max_values_tensor, max_indices.unsqueeze(0)))

   return max_values_tensor

print(clasificador(y_pred_multiclases))


tensor([4., 3.])
